# Dataset: Dermnet

## Contexto

Este conjunto de dados consiste em uma coleção de imagens representando 23 tipos distintos de doenças de pele. As imagens foram obtidas a partir do site [Dermnet](https://dermnetnz.org/topics/otitis-externa). O conjunto de dados contém aproximadamente 19.500 imagens, divididas em conjuntos de treino e teste, com cerca de 15.500 imagens no conjunto de treino e o restante no conjunto de teste.

## Conteúdo

As imagens estão no formato JPEG e possuem 3 canais (RGB). As resoluções das imagens variam, não sendo todas de alta resolução. As categorias das doenças incluem acne, melanoma, eczema, queratoses seborreicas, tinea (dermatofitose), doenças bolhosas, hera venenosa, psoríase, tumores vasculares, entre outras.

Este conjunto de dados oferece várias oportunidades de análise e aplicação, incluindo:

- Desenvolvimento de classificadores de imagens robustos para categorização automática de imagens em uma das 23 doenças identificadas.
- Análise exploratória das imagens para identificar diferenças visuais entre as categorias e fazer inferências sobre os padrões de cada doença.
- Agrupamento de imagens de diferentes doenças em categorias mais amplas para análise comparativa.
- Entre outras possibilidades de investigação e aplicação na área da dermatologia e reconhecimento de padrões.


In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from PIL import Image
import os
import pandas as pd
from glob import glob
import shutil

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Flatten, Dense, Dropout

In [3]:
SIZE = 250

In [4]:
INPUT_SHAPE = (SIZE, SIZE, 3) 

In [5]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

model = Sequential()
model.add(Conv2D(256, (3, 3), activation="relu", input_shape=(SIZE, SIZE, 3)))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3),activation='relu'))
#model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))  
model.add(Dropout(0.3))
model.add(Flatten())

model.add(Dense(32))
model.add(Dense(5, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 256)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 124, 124, 256)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 128)     295040    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 61, 61, 128)       0

In [6]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 30,
        width_shift_range=0.2,
        height_shift_range=0.2,
#         shear_range = 0.2,
#         zoom_range = 0.2,
#         horizontal_flip = True,
        fill_mode='reflect')

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# train_dir = '/kaggle/input/skin-disease-dataset/Dataset'
train_dir = '/Users/hugopereira/DatasetMEIA/train'
test_dir = '/Users/hugopereira/DatasetMEIA/test'

In [9]:
batch_size = 16

In [10]:
train_generator = train_datagen.flow_from_directory(
                                            directory = train_dir,
                                            class_mode = 'categorical',
                                            batch_size = batch_size,
                                            target_size = (SIZE, SIZE))

Found 2989 images belonging to 5 classes.


In [11]:
test_generator = test_datagen.flow_from_directory(
                                    directory = test_dir,
                                    class_mode = 'categorical',
                                    batch_size = batch_size,
                                    target_size = (SIZE, SIZE))

Found 749 images belonging to 5 classes.


In [16]:
model.fit(
        train_generator,
        steps_per_epoch=2000 // batch_size,   
        epochs=50,
#         validation_data=test_generator,
#         validation_steps=800 // batch_size
)

Epoch 1/50


2024-03-06 14:41:26.500560: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


  7/125 [>.............................] - ETA: 4:02 - loss: 1.0752 - acc: 0.5893

KeyboardInterrupt: 

In [19]:
model.evaluate(test_generator)

2024-03-01 12:06:33.542060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


251/251 [==============================] - 149s 595ms/step - loss: 2.9194 - acc: 0.1287


[2.919402599334717, 0.12868565320968628]